In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [2]:
mol_name = 'H2O'
# mol_name = 'H2'

# mol_name = 'LiH'

In [3]:
xyz_file = xyz_from_pubchem(mol_name)
print(xyz_file)

3
 
O	0	0	0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169



In [4]:
from symred.chem import Draw_molecule

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="stick")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
xyz_file = '4\n \nH\t0\t0\t0\nH\t1\t0\t0\nH\t2\t0\t0\nH\t3\t0\t0'

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="sphere")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
from symred.chem import PySCFDriver

In [7]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=50
ram = 8_000
run_mp2  = True,
run_cisd = True,
run_ccsd = True,
run_fci  = True

In [8]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [9]:
pyscf_obj.run_pyscf()

In [10]:
2*pyscf_obj.pyscf_hf.mol.nao

8

In [11]:
from symred.chem import FermionicHamilt

In [12]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

() 2.29310124732
((0, 1), (0, 0)) -1.8385126260957712
((0, 1), (4, 0)) 0.1535200531800066
((1, 1), (1, 0)) -1.8385126260957712
((1, 1), (5, 0)) 0.1535200531800066
((2, 1), (2, 0)) -1.5485198738611377
((2, 1), (6, 0)) 0.1346528866850458
((3, 1), (3, 0)) -1.5485198738611377
((3, 1), (7, 0)) 0.1346528866850458
((4, 1), (0, 0)) 0.15352005318000683
((4, 1), (4, 0)) -1.2423978239696827
((5, 1), (1, 0)) 0.15352005318000683
((5, 1), (5, 0)) -1.2423978239696827
((6, 1), (2, 0)) 0.13465288668504574
((6, 1), (6, 0)) -0.9084576464931442
((7, 1), (3, 0)) 0.13465288668504574
((7, 1), (7, 0)) -0.9084576464931442
((0, 1), (0, 1), (0, 0), (0, 0)) 0.2504327702741458
((0, 1), (0, 1), (0, 0), (4, 0)) -0.04165187456702536
((0, 1), (0, 1), (2, 0), (2, 0)) 0.07725837026056045
((0, 1), (0, 1), (2, 0), (6, 0)) -0.02286714657234941
((0, 1), (0, 1), (4, 0), (0, 0)) -0.041651874567025335
((0, 1), (0, 1), (4, 0), (4, 0)) 0.053084040051590224
((0, 1), (0, 1), (6, 0), (2, 0)) -0.022867146572349395
((0, 1), (0, 1), (

In [13]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 0, 0, 0, 0])

In [14]:
hf_ket = H_ferm.hf_ket
hf_ket.shape

(256,)

In [28]:
print(H_ferm.scf_method.energy_tot())
print(pyscf_obj.pyscf_hf.energy_tot())

H_mat = H_ferm.get_sparse_ham()
hf_ket.conj().T @ H_mat @ hf_ket

-2.0982818585784786
-2.0982818585784786


(-2.0982818585784786+0j)

In [15]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
e_orbs_occ = fij[:nocc]
e_i = e_orbs_occ.reshape(1, 1, -1, 1)
e_j = e_orbs_occ.reshape(1, 1, 1, -1)

e_orbs_vir = fij[nocc:]
e_a = e_orbs_vir.reshape(-1, 1, 1, 1)
e_b = e_orbs_vir.reshape(1, -1, 1, 1)

abgij = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (e_i + e_j - e_a - e_b)

## same spin - different spins
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

-0.04160571461928189

In [265]:
pyscf_obj.pyscf_mp2.e_corr

-0.04160571461928188

In [266]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

In [20]:
## key

# i,j,I,J are occupied molecular orbitals
# a,b,A,B are virtual molecular orbitals

# lower case = spin down !!!
# upper case = spin up   !!!

In [17]:
# opposite spins term
tIjAb=np.einsum('abij -> ijab', amplitudes, optimize='greedy')
tIjAb.shape

(2, 2, 2, 2)

In [18]:
# same spin term
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')
tijab.shape

(2, 2, 2, 2)

2

In [20]:
def alpha_occupied(i):
    return 2 * i

def alpha_unoccupied(i, n_alpha_electrons):
    return 2 * (i + n_alpha_electrons)

def beta_occupied(i):
    return 2 * i + 1

def beta_unoccupied(i, n_beta_electrons):
    return 2 * (i + n_beta_electrons) + 1

In [44]:
n_alpha, n_beta = pyscf_obj.pyscf_hf.mol.nelec

excite_op =FermionOperator()
for (i,j,a,b), value in np.ndenumerate(tIjAb):
    if abs(value)>1e-10:
        # 2 up 2 down
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)

        
for (i,j,b,a), value in np.ndenumerate(tijab):
    if abs(value)>1e-10:
        # 2 up 2 down
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)

len(list(excite_op))
# # excite_op
# excite_op

16

In [270]:
n_alpha, n_beta = pyscf_obj.pyscf_hf.mol.nelec

excite_op =FermionOperator()
for (a,b,i,j), value in np.ndenumerate(amplitudes):
    if abs(value)>1e-10:
        # all up
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)

#         # 3 up 1 down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)

        # 2 up 2 down
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)

#         # 1 up 3 down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)

        # all down
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
    
len(list(excite_op))
# excite_op

48

In [247]:
n_alpha, n_beta = pyscf_obj.pyscf_hf.mol.nelec

excite_op =FermionOperator()
for (a,b,i,j), value in np.ndenumerate(amplitudes):
    if abs(value)>1e-10:
#         all up
        if (alpha_unoccupied(a, n_alpha)!= alpha_unoccupied(b, n_beta) and alpha_occupied(i) != alpha_occupied(j) ):
            excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)

        # 2 up 2 down
#         if (alpha_unoccupied(a, n_alpha)!= alpha_unoccupied(b, n_beta) and beta_occupied(i) != beta_occupied(j) ):
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
#         if (alpha_unoccupied(a, n_alpha)!= beta_unoccupied(b, n_beta) and beta_occupied(i) != beta_occupied(j) ):
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
#         if (beta_unoccupied(a, n_alpha)!= beta_unoccupied(b, n_beta) and alpha_occupied(i) != alpha_occupied(j) ):
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)
#         if (beta_unoccupied(a, n_alpha)!= alpha_unoccupied(b, n_beta) and alpha_occupied(i) != beta_occupied(j) ):
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)

        # all down
        if (beta_unoccupied(a, n_alpha)!= beta_unoccupied(b, n_beta) and beta_occupied(i) != beta_occupied(j) ):
            excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
    
len(list(excite_op))
excite_op

-0.02891455886691224 [4^ 4^ 1 1] +
-0.05280759309647758 [4^ 4^ 3 3] +
-0.02891455886691224 [4^ 5^ 1 0] +
-0.05280759309647758 [4^ 5^ 3 2] +
-0.011627427169548121 [4^ 6^ 0 2] +
-0.011627427169548121 [4^ 6^ 1 3] +
-0.022405585894956455 [4^ 6^ 2 0] +
-0.022405585894956455 [4^ 6^ 3 1] +
-0.011627427169548121 [4^ 7^ 1 2] +
-0.022405585894956455 [4^ 7^ 3 0] +
-0.02891455886691224 [5^ 4^ 0 1] +
-0.05280759309647758 [5^ 4^ 2 3] +
-0.02891455886691224 [5^ 5^ 0 0] +
-0.05280759309647758 [5^ 5^ 2 2] +
-0.011627427169548121 [5^ 6^ 0 3] +
-0.022405585894956455 [5^ 6^ 2 1] +
-0.011627427169548121 [5^ 7^ 0 2] +
-0.011627427169548121 [5^ 7^ 1 3] +
-0.022405585894956455 [5^ 7^ 2 0] +
-0.022405585894956455 [5^ 7^ 3 1] +
-0.022405585894956476 [6^ 4^ 0 2] +
-0.022405585894956476 [6^ 4^ 1 3] +
-0.011627427169548123 [6^ 4^ 2 0] +
-0.011627427169548123 [6^ 4^ 3 1] +
-0.022405585894956476 [6^ 5^ 1 2] +
-0.011627427169548123 [6^ 5^ 3 0] +
-0.016527871022874615 [6^ 6^ 1 1] +
-0.020755795986627204 [6^ 6^ 3 3] +


In [271]:
counter=0
for a in range(2):
    for b in range(2):
        for i in range(2):
            for j in range(2):
                counter+=1
counter

16

In [30]:
mp2_ansatz = expm(get_sparse_operator(excite_op, n_qubits=H_ferm.n_qubits))
mp2_ket =  mp2_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ mp2_ket

(-2.1358714167943957+0j)

In [31]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

In [159]:
n_alpha, n_beta = pyscf_obj.pyscf_hf.mol.nelec

excite_op =FermionOperator()
for (a,b,i,j), value in np.ndenumerate(amplitudes):
    if abs(value)>1e-10:
        # all up
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)

#         # 3 up 1 down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)

        # 2 up 2 down
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)
#         # 1 up 3 down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)

        # all down
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
    
len(list(excite_op))


48

In [172]:
n_alpha, n_beta = pyscf_obj.pyscf_hf.mol.nelec

excite_op =FermionOperator()
for (a,b,i,j), value in np.ndenumerate(amplitudes):
    if abs(value)>1e-10:
        # all up
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)

#         # 3 up 1 down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)

        # 2 up 2 down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)
#         # 1 up 3 down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)

        # all down
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
    
len(list(excite_op))


40

In [177]:
n_alpha, n_beta = pyscf_obj.pyscf_hf.mol.nelec

excite_op =FermionOperator()
for (a,b,i,j), value in np.ndenumerate(amplitudes):
    if abs(value)>1e-10:
        # all up
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)

#         # 3 up 1 down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)

        # 2 up 2 down
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(j)} {alpha_occupied(i)}', value/2)
        excite_op+= FermionOperator(f'{alpha_unoccupied(b, n_alpha)}^ {beta_unoccupied(a, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{alpha_unoccupied(b, n_alpha)}^ {beta_unoccupied(a, n_alpha)}^ {beta_occupied(j)} {alpha_occupied(i)}', value/2)
        
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(j)} {alpha_occupied(i)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(b, n_alpha)}^ {alpha_unoccupied(a, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        excite_op+= FermionOperator(f'{beta_unoccupied(b, n_alpha)}^ {alpha_unoccupied(a, n_alpha)}^ {beta_occupied(j)} {alpha_occupied(i)}', value/2)
        
#         # 1 up 3 down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)

        # all down
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_alpha)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
    
len(list(excite_op))


16

In [141]:
# excite_op

In [179]:
mp2_ansatz = expm(get_sparse_operator(excite_op, n_qubits=H_ferm.n_qubits))
mp2_ket =  mp2_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ mp2_ket

(-2.0982818585784786+0j)

In [164]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

In [75]:
n_alpha, n_beta = pyscf_obj.pyscf_hf.mol.nelec

excite_op =FermionOperator()
for (a,b,i,j), value in np.ndenumerate(amplitudes):
    # up^ up^ up up
    excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {alpha_unoccupied(b, n_alpha)}^ {alpha_occupied(i)} {alpha_occupied(j)}', value/2)
    # down^ down^ down down
    excite_op+= FermionOperator(f'{beta_unoccupied(a, n_beta)}^ {beta_unoccupied(b, n_beta)}^ {beta_occupied(i)} {beta_occupied(j)}', value/2)
    
    # up^ down^ up down
    excite_op+= FermionOperator(f'{alpha_unoccupied(a, n_alpha)}^ {beta_unoccupied(b, n_beta)}^ {alpha_occupied(i)} {beta_occupied(j)}', value/2)
    # down^ up^ down up
    excite_op+= FermionOperator(f'{beta_unoccupied(a, n_beta)}^ {alpha_unoccupied(b, n_beta)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
    
    
    # up^ up^ down up
    excite_op+= FermionOperator(f'{beta_unoccupied(a, n_beta)}^ {alpha_unoccupied(b, n_beta)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        
    # down^ up^ down up
    excite_op+= FermionOperator(f'{beta_unoccupied(a, n_beta)}^ {alpha_unoccupied(b, n_beta)}^ {beta_occupied(i)} {alpha_occupied(j)}', value/2)
        
len(list(excite_op))


32

In [77]:
mp2_ansatz = expm(get_sparse_operator(0.25*excite_op, n_qubits=H_ferm.n_qubits))
mp2_ket =  mp2_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ mp2_ket

(-2.079989099020941+0j)

In [65]:
hf_ket.conj().T  @ mp2_ket

(1+0j)

In [67]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

In [22]:
tIjAb=tIjAb
tiJaB=tIjAb
tijab=tijab
tIJAB=tijab

In [25]:
def alpha_occupied(i):
    return 2 * i

def alpha_unoccupied(i, n_alpha_electrons):
    return 2 * (i + n_alpha_electrons)

def beta_occupied(i):
    return 2 * i + 1

def beta_unoccupied(i, n_beta_electrons):
    return 2 * (i + n_beta_electrons) + 1

In [24]:
# restricted=True

# threshold = 1e-12
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)


# for (I, J, A, B), value in np.ndenumerate(tIJAB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         I_occ = alpha_occupied(I)
#         J_occ = alpha_occupied(J)
#         A_vir = alpha_unoccupied(A, nocc)
#         B_vir = alpha_unoccupied(B, nocc)
#         excite_op+= FermionOperator(f'{A_vir}^ {I_occ} {B_vir}^ {J_occ}', value/2)
        
#         if (restricted):
#             # spin down
#             I_occ_d = beta_occupied(I)
#             J_occ_d = beta_occupied(J)
#             A_vir_d = beta_unoccupied(A, nocc)
#             B_vir_d = beta_unoccupied(B, nocc)
#             excite_op+= FermionOperator(f'{A_vir_d}^ {I_occ_d} {B_vir_d}^ {J_occ_d}', value/2)

# for (i, j, a, b), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#             i_occ = beta_occupied(i)
#             j_occ = beta_occupied(j)
#             a_vir = beta_unoccupied(a, nocc)
#             b_vir = beta_unoccupied(b, nocc)
#             excite_op+= FermionOperator(f'{a_vir}^ {i_occ} {b_vir}^ {j_occ}', value/2)
            
            
# for (I, j, A, b), value in np.ndenumerate(tIjAb):
#     if not np.isclose(value, 0.0, atol=threshold):
#         I_occ = alpha_occupied(I)
#         j_occ_d = beta_occupied(j)
#         A_vir = alpha_unoccupied(A, nocc)
#         b_vir_d = beta_unoccupied(b, nocc)
        
#         excite_op+= FermionOperator(f'{A_vir}^ {I_occ} {b_vir_d}^ {j_occ_d}', value/2)
        
#         if (restricted):
#             ## note tIjAb == tiJaB
#             i_occ_d = beta_occupied(I)
#             J_occ = alpha_occupied(j)
#             a_vir_d = beta_unoccupied(A, nocc)
#             B_vir = alpha_unoccupied(b, nocc)
#             excite_op+= FermionOperator(f'{a_vir_d}^ {i_occ_d} {B_vir}^ {J_occ}', value/2)

        
#             # Add missing same--spin amplitudes in restricted / closed-shell cases:
#             # T_IJAB = T_ijab = T_IjAb - T_IjBa
#             same_spin_amp = tIjAb[I,j,A,b] / 2. - tiJaB[I,j,b,A] / 2.
            
#             # alpha
#             excite_op+= FermionOperator(f'{A_vir}^ {I_occ} {B_vir}^ {J_occ}', same_spin_amp / 2)
            
#             # beta
#             excite_op+= FermionOperator(f'{b_vir_d}^ {j_occ_d} {a_vir_d}^ {i_occ_d}', same_spin_amp / 2)


# ######


# print(len(list(excite_op)))
# excite_op

In [25]:
# restricted=True

# threshold = 1e-12
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)


# for (i,j,a,b), value in np.ndenumerate(tIJAB):
#     # up,up,up,up
#     if not np.isclose(value, 0.0, atol=threshold):
#         # alpha
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                                       value/2)
        
#         if (restricted):
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                           value/2)

# for (i, j, a, b), value in np.ndenumerate(tijab):
#     # down,down,down,down
#     if not np.isclose(value, 0.0, atol=threshold):
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                       value/2)

    
# for (i, j, a, b), value in np.ndenumerate(tIjAb):
#     # up,down,up,down
#     if not np.isclose(value, 0.0, atol=threshold):
        
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                       value/2)
        
#         if (restricted):
#             # down,up ,up,down
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                                           value/2)

        
#             # Add missing same--spin amplitudes in restricted / closed-shell cases:
#             # T_IJAB = T_ijab = T_IjAb - T_IjBa
#             same_spin_amp = tIjAb[i,j,a,b] / 2. - tiJaB[i,j,b,a] / 2.
            
#             # alpha
#             excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               same_spin_amp/2)
            
#             # beta
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                               same_spin_amp/2)

            
# for (i, j, a, b), value in np.ndenumerate(tIJAB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               value/2)
        
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               value/2)
        
# #         if (restricted):
# #             ## note tIjAb == tiJaB
# #             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
# #                                           value/2)


# ######


# print(len(list(excite_op)))
# excite_op

In [26]:
# restricted=True

# threshold = 1e-12
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)


# for (i,j,a,b), value in np.ndenumerate(tIJAB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         # up,up,up,up
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                                       value/2)
        
#         if (restricted):
#             # down,down,down,down
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                           value/2)

# for (i, j, a, b), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         # down,down,down,down
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                       value/2)

    
# for (i, j, a, b), value in np.ndenumerate(tIjAb):
#     if not np.isclose(value, 0.0, atol=threshold):
#         # up, up, down, down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                       value/2)
        
#         if (restricted):
#             # down, down, up, up
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                                           value/2)

        
#             # Add missing same--spin amplitudes in restricted / closed-shell cases:
#             # T_IJAB = T_ijab = T_IjAb - T_IjBa
#             same_spin_amp = tIjAb[i,j,a,b] / 2. - tiJaB[i,j,b,a] / 2.
            
#             # up up, up, up
#             excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               same_spin_amp/2)
            
#             # down, down, down, down
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                               same_spin_amp/2)

            
# for (i, j, a, b), value in np.ndenumerate(tIJAB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         # up, down, up, up
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               value/2)
        
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               value/2)
        
# #         if (restricted):
# #             ## note tIjAb == tiJaB
# #             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
# #                                           value/2)


# ######


# print(len(list(excite_op)))
# excite_op

In [27]:
restricted=True

threshold = 1e-12
excite_op =FermionOperator()
nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)


for (i,j,a,b), value in np.ndenumerate(tIJAB):
    # up,up,up,up
    if not np.isclose(value, 0.0, atol=threshold):
        # up,up,up,up
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
                                      value/2)
        
        if (restricted):
            # down,down,down,down
            excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
                                          value/2)

for (i, j, a, b), value in np.ndenumerate(tijab):
    # down,down,down,down
    if not np.isclose(value, 0.0, atol=threshold):
        # up,up,up,up
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
                                      value/2)

    
for (i, j, a, b), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=threshold):
        # up, up, down, down
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
                                      value/2)
        
        if (restricted):
            # down, down, up, up
            excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
                                          value/2)

        
            # Add missing same--spin amplitudes in restricted / closed-shell cases:
            # T_IJAB = T_ijab = T_IjAb - T_IjBa
            same_spin_amp = tIjAb[i,j,a,b] / 2. - tiJaB[i,j,b,a] / 2.
            
            # up up, up, up
            excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
                              same_spin_amp/2)
            
            # down, down, down, down
            excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
                              same_spin_amp/2)



######


print(len(list(excite_op)))
excite_op

24


-0.02891455886691224 [4^ 0 5^ 1] +
0.026945396813520886 [4^ 0 6^ 2] +
-0.011627427169548121 [4^ 0 7^ 3] +
-0.05280759309647758 [4^ 2 5^ 3] +
-0.02694539681352083 [4^ 2 6^ 0] +
-0.022405585894956455 [4^ 2 7^ 1] +
-0.02891455886691224 [5^ 1 4^ 0] +
-0.011627427169548121 [5^ 1 6^ 2] +
0.01616723808811253 [5^ 1 7^ 3] +
-0.05280759309647758 [5^ 3 4^ 2] +
-0.022405585894956455 [5^ 3 6^ 0] +
-0.016167238088112497 [5^ 3 7^ 1] +
-0.026945396813520886 [6^ 0 4^ 2] +
-0.022405585894956476 [6^ 0 5^ 3] +
-0.016527871022874615 [6^ 0 7^ 1] +
0.02694539681352083 [6^ 2 4^ 0] +
-0.011627427169548123 [6^ 2 5^ 1] +
-0.020755795986627204 [6^ 2 7^ 3] +
-0.022405585894956476 [7^ 1 4^ 2] +
-0.01616723808811253 [7^ 1 5^ 3] +
-0.016527871022874615 [7^ 1 6^ 0] +
-0.011627427169548123 [7^ 3 4^ 0] +
0.016167238088112497 [7^ 3 5^ 1] +
-0.020755795986627204 [7^ 3 6^ 2]

In [29]:
# double_amplitudes.shape

In [30]:
# Store doubles, include factor of 1/2 for convention
T2IJAB_Amps = tIJAB
T2ijab_Amps = tijab
T2IjAb_Amps = tIjAb

double_amplitudes = np.zeros((2*pyscf_obj.pyscf_hf.mol.nao, ) * 4)

for (i, j, a, b), value in np.ndenumerate(T2IJAB_Amps):
    double_amplitudes[alpha_unoccupied(a),
                      alpha_occupied(i),
                      alpha_unoccupied(b),
                      alpha_occupied(j)] = value / 2.
    if (restricted):
        double_amplitudes[beta_unoccupied(a),
                          beta_occupied(i),
                          beta_unoccupied(b),
                          beta_occupied(j)] = value / 2.

for (i, j, a, b), value in np.ndenumerate(T2ijab_Amps):
    double_amplitudes[beta_unoccupied(a),
                      beta_occupied(i),
                      beta_unoccupied(b),
                      beta_occupied(j)] = value / 2.

for (i, j, a, b), value in np.ndenumerate(T2IjAb_Amps):
    double_amplitudes[alpha_unoccupied(a),
                      alpha_occupied(i),
                      beta_unoccupied(b),
                      beta_occupied(j)] = value / 2.

    if (restricted):
        double_amplitudes[beta_unoccupied(a),
                          beta_occupied(i),
                          alpha_unoccupied(b),
                          alpha_occupied(j)] = value / 2.

        # Add missing same--spin amplitudes in restricted / closed-shell cases:
        # T_IJAB = T_ijab = T_IjAb - T_IjBa

        same_spin_amp = T2IjAb_Amps[i,j,a,b] / 2. - T2IjAb_Amps[i,j,b,a] / 2.

        double_amplitudes[alpha_unoccupied(a),
                          alpha_occupied(i),
                          alpha_unoccupied(b),
                          alpha_occupied(j)] = same_spin_amp / 2.

        double_amplitudes[beta_unoccupied(a),
                          beta_occupied(i),
                          beta_unoccupied(b),
                          beta_occupied(j)] = same_spin_amp / 2.


TypeError: alpha_unoccupied() missing 1 required positional argument: 'n_alpha_electrons'

In [31]:
print(len(list(generator)))
generator

NameError: name 'generator' is not defined

In [ ]:
#    # Store doubles, include factor of 1/2 for convention
#     for entry in T2IJAB_Amps:
#         i, j, a, b, value = entry
#         double_amplitudes[alpha_unoccupied(a),
#                           alpha_occupied(i),
#                           alpha_unoccupied(b),
#                           alpha_occupied(j)] = value / 2.
#         if (restricted):
#             double_amplitudes[beta_unoccupied(a),
#                               beta_occupied(i),
#                               beta_unoccupied(b),
#                               beta_occupied(j)] = value / 2.

#     for entry in T2ijab_Amps:
#         i, j, a, b, value = entry
#         double_amplitudes[beta_unoccupied(a),
#                           beta_occupied(i),
#                           beta_unoccupied(b),
#                           beta_occupied(j)] = value / 2.

#     for entry in T2IjAb_Amps:
#         i, j, a, b, value = entry
#         double_amplitudes[alpha_unoccupied(a),
#                           alpha_occupied(i),
#                           beta_unoccupied(b),
#                           beta_occupied(j)] = value / 2.

#         if (restricted):
#             double_amplitudes[beta_unoccupied(a),
#                               beta_occupied(i),
#                               alpha_unoccupied(b),
#                               alpha_occupied(j)] = value / 2.

In [ ]:
# threshold = 1e-8
# variables = {}
# if tIjAb is not None:
#     nvirt = tIjAb.shape[2]
#     nocc = tIjAb.shape[0]
#     assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

#     for (I, j, A, b), value in np.ndenumerate(tIjAb):
#         if not np.isclose(value, 0.0, atol=threshold):
#             variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
#     for (i, J, a, B), value in np.ndenumerate(tiJaB):
#         if not np.isclose(value, 0.0, atol=threshold):
#             variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
#     for (i, j, a, b), value in np.ndenumerate(tijab):
#         if not np.isclose(value, 0.0, atol=threshold):
#             variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
#     for (I, J, A, B), value in np.ndenumerate(tIJAB):
#         if not np.isclose(value, 0.0, atol=threshold):
#             variables[(2 * (nocc + A), 2 * I, 2 * (nocc + B), J)] = value

# print(len(variables))
# variables

In [ ]:
generator

In [32]:
mp2_ansatz = expm(get_sparse_operator(0.5*excite_op, n_qubits=H_ferm.n_qubits))
## mp2_ansatz = expm(get_sparse_operator(excite_op, n_qubits=H_ferm.n_qubits))

## help(qubit_operator_sparse)

In [34]:
mp2_ket =  mp2_ansatz @ hf_ket
print(np.allclose(mp2_ket, hf_ket))
hf_ket.conj().T @ H_mat @ mp2_ket

False


(-2.1130604812830724+0j)

In [33]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

In [ ]:
# tIjAb=np.einsum('abij -> abij', amplitudes, optimize='greedy')

# tIjAb.shape

In [ ]:
# pyscf_obj.pyscf_mp2.t2 # <-- these are the amplitudes!

In [35]:
# variables = {}
# if tIjAb is not None:
#     nvirt = tIjAb.shape[2]
#     nocc = tIjAb.shape[0]
#     assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
#     for (I, J, A, B), value in np.ndenumerate(tIjAb):
#         if not np.isclose(value, 0.0, atol=1e-8):
#             variables[(nocc + A, I, nocc + B, J)] = value
            

nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
excite_op =FermionOperator()
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
for (I, J, A, B), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=1e-8):
        excite_op+= FermionOperator(f'{nocc + A}^ {I}^ {nocc + B} {J}', value)
#         excite_op+= FermionOperator(f'{I}^ {J}^ {nocc + A} {nocc + B}', value)
#         excite_op+= FermionOperator(f'{nocc + A}^ {nocc + B}^ {I} {J}', -value)

In [36]:
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

nvirt = tijab.shape[2]
nocc = tijab.shape[0]
excite_op =FermionOperator()
assert (tijab.shape[1] == nocc and tijab.shape[3] == nvirt)
for (I, J, A, B), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=1e-8):
        excite_op+= FermionOperator(f'{nocc + A}^ {I}^ {nocc + B} {J}', value)

In [37]:
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

tIjAb=tIjAb
tiJaB=tIjAb
tijab=tijab
tIJAB=tijab


threshold = 1e-8
excite_op =FermionOperator()
nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

for (I, j, A, b), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + A)}^ {2*I}^ {2 * (nocc + b) + 1} {j + 1}', value/2)
#         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
for (i, J, a, B), value in np.ndenumerate(tiJaB):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + a)+1}^ {2*i+1}^ {2 * (nocc + B)} {J}', value/2)
#         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
for (i, j, a, b), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + a)+1}^ {2*i+1}^ {2 * (nocc + b)+1} {j+1}', value/2)
#         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
for (I, J, A, B), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=threshold):
#         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + B), J)] = value
        excite_op+= FermionOperator(f'{2 * (nocc + A)}^ {2*I}^ {2 * (nocc + B)} {J}', value/2)

In [ ]:
# tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

# tIjAb=tIjAb
# tiJaB=tIjAb
# tijab=tijab
# tIJAB=tijab


# threshold = 1e-8
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

# for (I, j, A, b), value in np.ndenumerate(tIjAb):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + A), 2*I, 2 * (nocc + b) + 1, j + 1]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
# for (i, J, a, B), value in np.ndenumerate(tiJaB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + a)+1, 2*i+1, 2 * (nocc + B), J]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
# for (i, j, a, b), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + a)+1, 2*i+1, 2 * (nocc + b)+1, j+1]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
# for (I, J, A, B), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + A), 2*I, 2 * (nocc + B), J]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)

In [ ]:
len(list(excite_op))

# excite_op/2

In [ ]:
len(list(generator))

In [ ]:
generator

In [ ]:
mp2_ansatz = expm(get_sparse_operator(0.25*excite_op, n_qubits=H_ferm.n_qubits))
## mp2_ansatz = expm(get_sparse_operator(excite_op, n_qubits=H_ferm.n_qubits))

## help(qubit_operator_sparse)

In [ ]:
mp2_ket =  mp2_ansatz @ hf_ket
mp2_ket.conj().T @ H_mat @ mp2_ket

In [ ]:
pyscf_obj.pyscf_mp2.e_tot

In [ ]:
pyscf_obj.pyscf_mp2.e_hf

In [ ]:
hf_ket.conj().T @ mp2_ket

In [ ]:
# t(a,i,b,j) = 0.25 * g(a,i,b,j)/(e(i) + e(j) -a(i) - b(j) )

# +aibj
# -abij

In [ ]:
pyscf_obj.pyscf_mp2.e_corr #+ pyscf_obj.pyscf_mp2.e_corr

In [38]:
variables = {}
if tIjAb is not None:
    nvirt = tIjAb.shape[2]
    nocc = tIjAb.shape[0]
    assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
    for (I, J, A, B), value in np.ndenumerate(tIjAb):
        if not np.isclose(value, 0.0, atol=1e-8):
            variables[(nocc + A, I, nocc + B, J)] = value

dict(sorted(variables.items(), key=lambda x: np.abs(x[1]), reverse=True))

{(2, 1, 2, 1): -0.10561518619295517,
 (2, 0, 2, 0): -0.05782911773382448,
 (3, 0, 2, 1): -0.04481117178991295,
 (2, 1, 3, 0): -0.04481117178991291,
 (3, 1, 3, 1): -0.04151159197325441,
 (3, 0, 3, 0): -0.03305574204574923,
 (3, 1, 2, 0): -0.023254854339096246,
 (2, 0, 3, 1): -0.023254854339096243}

In [274]:
from pyscf.cc.addons import spatial2spin

t2 = spatial2spin(pyscf_obj.pyscf_mp2.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [275]:
# double_amplitudes_list=[]
# double_amplitudes = double_amps
# for i, j, k, l in zip(*double_amplitudes.nonzero()):
#     double_amplitudes_list.append([[i, j, k, l],
#                                    double_amplitudes[i, j, k, l]])
    
# double_amplitudes_list

double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])

print(len(double_amplitudes_list))
double_amplitudes_list

40


[[[4, 0, 5, 1], -0.028914558866912243],
 [[4, 0, 6, 2], -0.010778158725408358],
 [[4, 0, 7, 3], -0.022405585894956476],
 [[4, 1, 5, 0], 0.028914558866912243],
 [[4, 1, 7, 2], 0.011627427169548121],
 [[4, 2, 5, 3], -0.05280759309647757],
 [[4, 2, 6, 0], 0.010778158725408347],
 [[4, 2, 7, 1], -0.011627427169548121],
 [[4, 3, 5, 2], 0.05280759309647757],
 [[4, 3, 7, 0], 0.022405585894956476],
 [[5, 0, 4, 1], 0.028914558866912243],
 [[5, 0, 6, 3], 0.011627427169548118],
 [[5, 1, 4, 0], -0.028914558866912243],
 [[5, 1, 6, 2], -0.02240558589495647],
 [[5, 1, 7, 3], -0.010778158725408358],
 [[5, 2, 4, 3], 0.05280759309647757],
 [[5, 2, 6, 1], 0.02240558589495647],
 [[5, 3, 4, 2], -0.05280759309647757],
 [[5, 3, 6, 0], -0.011627427169548118],
 [[5, 3, 7, 1], 0.010778158725408347],
 [[6, 0, 4, 2], 0.010778158725408358],
 [[6, 0, 5, 3], -0.011627427169548118],
 [[6, 0, 7, 1], -0.016527871022874608],
 [[6, 1, 5, 2], 0.02240558589495647],
 [[6, 1, 7, 0], 0.016527871022874608],
 [[6, 2, 4, 0], -0.0

In [276]:
generator = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [181]:
len(list(generator))

40

In [277]:
mp2_ansatz = expm(get_sparse_operator(0.25*generator, n_qubits=H_ferm.n_qubits))
# mp2_ansatz = expm(get_sparse_operator(0.125*generator, n_qubits=H_ferm.n_qubits))

In [278]:
mp2_ket =  mp2_ansatz @ hf_ket
mp2_ket.conj().T @ H_mat @ mp2_ket

(-2.145791117472204+0j)

In [279]:
hf_ket.conj().T @ H_mat @ mp2_ket

(-2.1190847158881194+0j)

In [280]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

(1.005840375776513+0j)

In [48]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
ei = fij[:nocc]
ai = fij[nocc:]
V = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (
        ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

-0.04160571461928189

https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.154.2423&rep=rep1&type=pdf

In [89]:
mp2_ansatz = expm(get_sparse_operator(0.5*generator, n_qubits=H_ferm.n_qubits))
# mp2_ansatz = expm(get_sparse_operator(0.125*generator, n_qubits=H_ferm.n_qubits))

In [90]:
mp2_ket =  mp2_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ mp2_ket

(-2.1398875731977607+0j)

In [96]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

In [91]:
(mp2_ket.conj().T  @ hf_ket) # "intermediate normalization!"

(1+0j)

In [92]:
pyscf_obj.pyscf_mp2.e_corr

-0.04160571461928188

In [94]:
(hf_ket.conj().T @ H_mat @ mp2_ket - pyscf_obj.pyscf_mp2.e_hf).real

-0.041605714619282086

In [95]:
# https://www.ipc.kit.edu/theochem/download/Kapitel3.pdf pg8!!!
H_new =  np.linalg.inv(mp2_ansatz.todense()) @ H_mat @ mp2_ansatz

(hf_ket.conj().T @ H_new @ hf_ket)

matrix([[-2.13988757+0.j]])

In [123]:
H_new =  H_mat @ mp2_ansatz.todense()

In [124]:
eigvals, eigvecs = np.linalg.eig(H_new)

In [125]:
idx = eigvals.argsort() 
eigvals = eigvals[idx].real
eigvecs = eigvecs[:,idx]

eigvecs[:,0].conj().T @ eigvecs[:,0]

matrix([[1.+0.j]])

In [127]:
state = eigvecs[:,0]

mp2_state_energy = (state.conj().T @ H_mat @ state).real
mp2_state_energy

matrix([[-2.14395524]])

In [106]:
mp2_ket_norm = mp2_ket/np.linalg.norm(mp2_ket)
(mp2_ket_norm.conj().T @ H_mat @ mp2_ket_norm).real

-2.1549414883264264

In [107]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

In [100]:
hf_ket.conj().T @ mp2_ket

(1+0j)

In [105]:
new_ket = mp2_ket + hf_ket
new_ket = new_ket/np.linalg.norm(new_ket)

(new_ket.conj().T @ H_mat @ new_ket).real

-2.1333758951620374

In [286]:
hf_ket.conj().T @ mp2_ket

(1+0j)

In [295]:
overlap_mp2 = (mp2_ket.conj().T @ mp2_ket).real
C0 = 1/np.sqrt(overlap_mp2)

norm_state = (mp2_ket) * C0

norm_state.conj().T @ norm_state

(1.0000000000000002+0j)

In [298]:
(norm_state.conj().T @ H_mat @ norm_state).real

-2.1333316589281317

https://mattermodeling.stackexchange.com/questions/1540/how-to-calculate-the-fock-matrix-in-the-molecular-orbital-basis-pyscf

In [51]:
pyscf_obj.pyscf_hf.converged

True

In [80]:
# Szabo eq: 6.65

v_hf =  np.einsum('ibjb->ij', H_ferm._two_body_integrals)  - np.einsum('ibbj->ij', H_ferm._two_body_integrals)

Fock_mo = H_ferm._one_body_integrals + v_hf


In [84]:
from openfermion import InteractionOperator

core_constant=0
Fock_op = InteractionOperator(core_constant,
                              Fock_mo,
                              np.zeros_like(H_ferm._two_body_integrals))

Fock_mo_mat = get_sparse_operator(Fock_op, n_qubits=H_ferm.n_qubits)

(hf_ket.conj().T @ Fock_mo_mat @ hf_ket).real
# Fock_mo_mat

ValueError: dimension mismatch

In [ ]:
hf_ket.conj().T @ H_mat @ mp2_ket

In [ ]:
mp2_ss_corr = 0.0
for i in xrange(ndocc):
    for a in xrange(nmo - ndocc):
        for j in xrange(ndocc):
            for b in xrange(nmo - ndocc):
                numerator = I_mo[i,a,j,b] * (I_mo[i, a, j, b] - I_mo[i, b, j, a])
                mp2_ss_corr += numerator / (e_ij[i] + e_ij[j] - e_ab[a] - e_ab[b])

In [309]:
double_occ

2

In [313]:
V = np.zeros_like(two_body_integrals)
double_occ = H_ferm.n_electrons
double_vir = H_ferm.n_qubits- double_occ
for i in range(double_occ):
    for a in range(double_vir):
        for j in range(double_occ):
            for b in range(double_vir):
                V[i,a,j,b] = two_body_integrals[i,a,j,b]

                

True